### Text Summerizer

Library

In [1]:
import spacy
from heapq import nlargest
from collections import Counter
from string import punctuation
from spacy.lang.en.stop_words import STOP_WORDS


In [2]:
import spacy.cli
spacy.cli.download("en_core_web_lg")
import en_core_web_lg
nlp = en_core_web_lg.load()

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [3]:
text='''
Sex is the trait that determines whether a sexually reproducing organism produces male or female gametes.[1][2][3] During sexual reproduction, a male and a female gamete fuse to form a zygote, which develops into an offspring that inherits traits from each parent. By convention, organisms that produce smaller, more mobile gametes (spermatozoa, sperm) are called male, while organisms that produce produce larger, non-mobile gametes (ova, often called egg cells) are called female.[4] An organism, such as most flowering plants, that produces both types of gamete is a hermaphrodite.[3][5]

The male and female of a species may be physically alike (sexual monomorphism) or have physical differences (sexual dimorphism) that reflect various reproductive pressures on each sex. Sexual selection or mate choice can accelerate the evolution of differences between the sexes.

The terms 'male' and 'female' typically do not apply in sexually undifferentiated species in which the individuals are isomorphic (look the same) and the gametes are isogamous (indistinguishable in size and shape), such as the green alga Ulva lactuca. Some kinds of functional differences between individuals, such as in fungi,[6] may be referred to as mating types.[7]

There are several sex-determination systems. Most mammalian species have the XY sex-determination system, where the male usually carries an X and a Y chromosome (XY), and the female usually carries two X chromosomes (XX). Other chromosomal sex-determination systems in animals include the ZW system in birds, and the XO system in insects. Various environmental systems include temperature-dependent sex determination in reptiles and crustaceans.[8]
'''

In [4]:
document=nlp(text)
keywords=[]
stopwords=list(STOP_WORDS)
pos_tag=['NOUN','VERB','ADJ','PROPN']
for token in document:
   if (token.text in stopwords or token.text in punctuation):
     continue
   if (token.pos_ in pos_tag):
     keywords.append(token.text)


In [5]:
freq_words=Counter(keywords)
freq_words.most_common(6)

[('male', 6),
 ('female', 5),
 ('sex', 5),
 ('determination', 4),
 ('sexual', 3),
 ('produce', 3)]

In [6]:
max_freq_words=Counter(keywords).most_common(1)[0][1]
max_freq_words

6

In [7]:
for word in freq_words.keys():
  freq_words[word]=freq_words[word]/max_freq_words
freq_words.most_common(6)

[('male', 1.0),
 ('female', 0.8333333333333334),
 ('sex', 0.8333333333333334),
 ('determination', 0.6666666666666666),
 ('sexual', 0.5),
 ('produce', 0.5)]

In [8]:
sent_str={}
for sent in document.sents:
  for word in sent:
    if word.text in freq_words.keys():
      if sent in sent_str.keys():
        sent_str[sent]+=freq_words[word.text]
      else:
        sent_str[sent]=freq_words[word.text]
print(sent_str)

{
Sex is the trait that determines whether a sexually reproducing organism produces male or female gametes.[1][2][3]: 3.1666666666666665, During sexual reproduction, a male and a female gamete fuse to form a zygote, which develops into an offspring that inherits traits from each parent.: 4.166666666666666, By convention, organisms that produce smaller, more mobile gametes (spermatozoa, sperm) are called male, while organisms that produce produce larger, non-mobile gametes (ova, often called egg cells) are called female.[4]: 7.666666666666667, An organism, such as most flowering plants, that produces both types of gamete is a hermaphrodite.[3][5]

The male and female of a species may be physically alike (sexual monomorphism) or have physical differences (sexual dimorphism) that reflect various reproductive pressures on each sex.: 7.333333333333335, Sexual selection or mate choice can accelerate the evolution of differences between the sexes.

: 1.6666666666666667, The terms 'male' and '

In [9]:
summerized=nlargest(3, sent_str, key=sent_str.get)
print(' '.join([m.text for m in summerized]))

By convention, organisms that produce smaller, more mobile gametes (spermatozoa, sperm) are called male, while organisms that produce produce larger, non-mobile gametes (ova, often called egg cells) are called female.[4] An organism, such as most flowering plants, that produces both types of gamete is a hermaphrodite.[3][5]

The male and female of a species may be physically alike (sexual monomorphism) or have physical differences (sexual dimorphism) that reflect various reproductive pressures on each sex. Most mammalian species have the XY sex-determination system, where the male usually carries an X and a Y chromosome (XY), and the female usually carries two X chromosomes (XX).


In [10]:
from transformers import BartTokenizer, BartForConditionalGeneration
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
input_text=text
input_ids=tokenizer.batch_encode_plus (
    [input_text],
    return_tensors='pt',
    max_length=1024,
    truncation=True

)
summary_ids=model.generate(
    input_ids['input_ids'],
    num_beams=4,
    max_length=100,
    early_stopping=True

)
summary=tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
print('Summary:', summary)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Summary: Sex is the trait that determines whether a sexually reproducing organism produces male or female gametes. During sexual reproduction, a male and a female gamete fuse to form a zygote, which develops into an offspring that inherits traits from each parent. Most mammalian species have the XY sex-determination system, where the male usually carries an X and a Y chromosome.
